In [ ]:
# Mettre à jour la liste des paquets disponibles
!apt-get update

# Installer Chromium et Chromedriver
!apt-get install -y chromium-chromedriver

# Installer les bibliothèques Python nécessaires
!pip install selenium beautifulsoup4 pandas

# Copier Chromedriver dans le répertoire approprié
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Importer les bibliothèques nécessaires
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Configurer le webdriver (assurez-vous d'avoir le driver correct pour votre navigateur)
driver = webdriver.Chrome()

# URL du site web
# pour avoir les startups à succès :
#url = "https://www.failory.com/interviews?type-of-interview=Successful%20startup"
# pour avoir les startups échouées :
url = "https://www.failory.com/interviews?type-of-interview=Failed%20startup"
driver.get(url)

# Faire défiler jusqu'en bas pour charger toutes les données
SCROLL_PAUSE_TIME = 2

# Obtenir la hauteur de défilement
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Faire défiler vers le bas de la page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Attendre que la page se charge
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculer la nouvelle hauteur de défilement et comparer avec l'ancienne hauteur de défilement
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Analyser la source de la page avec BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Trouver toutes les listes de startups
listings = soup.select('div.w-dyn-items > div')

# Liste pour stocker toutes les données extraites
data = []

for listing in listings:
    problem = listing.select_one('a > div.card-title').get_text(strip=True)
    sector = listing.select_one('div.interviews-filters-div-block.w-clearfix > div:nth-child(3)').get_text(strip=True)
    country = listing.select_one('div.interviews-filters-div-block.w-clearfix > div:nth-child(4)').get_text(strip=True)
    reason = listing.select_one('div.interviews-filters-div-block.w-clearfix > div:nth-child(5)').get_text(strip=True)
    profit = listing.select_one('div.interviews-filters-div-block.w-clearfix > div:nth-child(6)').get_text(strip=True)
    founder = listing.select_one('div.interviews-card-data-div-block > div > div.interviews-card-name').get_text(strip=True)
    date = listing.select_one('div.interviews-card-data-div-block > div > div.card-date').get_text(strip=True)
    link = listing.select_one('a')['href']
    startup_name = link.split('/')[-1]

    # Ouvrir la page de détails
    driver.get(f"https://www.failory.com{link}")
    details_soup = BeautifulSoup(driver.page_source, 'html.parser')
    summary = details_soup.select_one('#content > p').get_text(strip=True)

    # Stocker les données extraites
    data.append({
        'Problème': problem,
        'Secteur': sector,
        'Pays': country,
        'Raison': reason,
        'Profit': profit,
        'Fondateur': founder,
        'Date': date,
        'Nom de la startup': startup_name,
        'Résumé': summary
    })

    # Retourner à la page principale
    driver.back()

# Convertir en DataFrame
df = pd.DataFrame(data)
df.to_csv('startup_failed.csv', index=False)

# Fermer le driver
driver.quit()

print("Extraction terminée. Les données sont sauvegardées dans startup_failed.csv.")